In [1]:
import pandas as pd
import sqlite3

In [2]:
from google.colab import files
uploaded = files.upload()

Saving database.sqlite to database (1).sqlite


In [3]:
conn = sqlite3.connect('database.sqlite')  # Remplace par le nom exact du fichier téléchargé
cursor = conn.cursor()

In [4]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables dans la base de données :")
print(tables)

Tables dans la base de données :
               name
0            Player
1        Extra_Runs
2    Batsman_Scored
3     Batting_Style
4     Bowling_Style
5           Country
6            Season
7              City
8           Outcome
9            Win_By
10     Wicket_Taken
11            Venue
12       Extra_Type
13         Out_Type
14    Toss_Decision
15           Umpire
16             Team
17     Ball_by_Ball
18      sysdiagrams
19  sqlite_sequence
20            Match
21            Rolee
22     Player_Match


In [5]:
table_names = tables['name'].tolist()

# Afficher les colonnes pour chaque table
for table in table_names:
    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn)
    print(f"\n📋 Table : {table}")
    print(f"Colonnes : {df.columns.tolist()}")


📋 Table : Player
Colonnes : ['Player_Id', 'Player_Name', 'DOB', 'Batting_hand', 'Bowling_skill', 'Country_Name']

📋 Table : Extra_Runs
Colonnes : ['Match_Id', 'Over_Id', 'Ball_Id', 'Extra_Type_Id', 'Extra_Runs', 'Innings_No']

📋 Table : Batsman_Scored
Colonnes : ['Match_Id', 'Over_Id', 'Ball_Id', 'Runs_Scored', 'Innings_No']

📋 Table : Batting_Style
Colonnes : ['Batting_Id', 'Batting_hand']

📋 Table : Bowling_Style
Colonnes : ['Bowling_Id', 'Bowling_skill']

📋 Table : Country
Colonnes : ['Country_Id', 'Country_Name']

📋 Table : Season
Colonnes : ['Season_Id', 'Man_of_the_Series', 'Orange_Cap', 'Purple_Cap', 'Season_Year']

📋 Table : City
Colonnes : ['City_Id', 'City_Name', 'Country_id']

📋 Table : Outcome
Colonnes : ['Outcome_Id', 'Outcome_Type']

📋 Table : Win_By
Colonnes : ['Win_Id', 'Win_Type']

📋 Table : Wicket_Taken
Colonnes : ['Match_Id', 'Over_Id', 'Ball_Id', 'Player_Out', 'Kind_Out', 'Fielders', 'Innings_No']

📋 Table : Venue
Colonnes : ['Venue_Id', 'Venue_Name', 'City_Id']

📋

In [6]:
# Exécuter la requête SQL pour afficher toutes les colonnes de Player_Match
query = "SELECT * FROM Player_Match;"
df_player_match = pd.read_sql_query(query, conn)

# Afficher les 5 premières lignes pour un aperçu
df_player_match.head()

,Match_Id,Player_Id,Role_Id,Team_Id
0,335987,1,1,1
1,335987,2,3,1
2,335987,3,3,1
3,335987,4,3,1
4,335987,5,3,1


In [11]:
query="""
SELECT p.Player_Name, SUM(bs.Runs_Scored) AS total_runs
FROM Batsman_Scored bs
JOIN Ball_by_Ball bbb
ON bs.Match_Id = bbb.Match_Id
AND bs.Over_Id = bbb.Over_Id
AND bs.Ball_Id = bbb.Ball_Id
JOIN Player p
ON bbb.Striker = p.Player_Id
GROUP BY p.Player_Name
ORDER BY total_runs DESC;
"""

df_batsman_runs = pd.read_sql_query(query, conn)
df_batsman_runs.head(10)  # Affiche les 10 meilleurs batteurs

,Player_Name,total_runs
0,V Kohli,8158
1,SK Raina,7588
2,RG Sharma,7263
3,G Gambhir,6793
4,RV Uthappa,6303
5,DA Warner,6235
6,CH Gayle,6105
7,MS Dhoni,6006
8,S Dhawan,5917
9,AB de Villiers,5832


In [12]:
query = """
WITH Batsman_Match_Scores AS (
    SELECT
        bbb.Striker,
        bs.Match_Id,
        SUM(bs.Runs_Scored) AS runs_in_match
    FROM Batsman_Scored bs
    JOIN Ball_by_Ball bbb
      ON bs.Match_Id = bbb.Match_Id
     AND bs.Over_Id = bbb.Over_Id
     AND bs.Ball_Id = bbb.Ball_Id
    GROUP BY bbb.Striker, bs.Match_Id
)

SELECT
    p.Player_Name,
    COUNT(CASE WHEN bms.runs_in_match BETWEEN 50 AND 99 THEN 1 END) AS fifties,
    COUNT(CASE WHEN bms.runs_in_match >= 100 THEN 1 END) AS hundreds
FROM Batsman_Match_Scores bms
JOIN Player p ON bms.Striker= p.Player_Id
GROUP BY p.Player_Name
ORDER BY hundreds DESC, fifties DESC;
"""

# Exécuter la requête dans pandas
df_fifties_hundreds = pd.read_sql_query(query, conn)
df_fifties_hundreds.head(10)

,Player_Name,fifties,hundreds
0,V Kohli,29,32
1,DA Warner,21,28
2,G Gambhir,32,24
3,CH Gayle,19,24
4,AM Rahane,18,21
5,RG Sharma,36,20
6,SK Raina,44,19
7,AB de Villiers,29,19
8,SE Marsh,15,19
9,RV Uthappa,40,15


In [13]:
query = """
SELECT
    p.Player_Name,
    bbb.Match_Id,
    COUNT(wt.Player_Out) AS Wickets,
    SUM(COALESCE(bs.Runs_Scored, 0) + COALESCE(er.Extra_Runs, 0)) AS Runs_Conceded
FROM Ball_by_Ball bbb
LEFT JOIN Wicket_Taken wt
    ON bbb.Match_Id = wt.Match_Id
    AND bbb.Over_Id = wt.Over_Id
    AND bbb.Ball_Id = wt.Ball_Id
LEFT JOIN Batsman_Scored bs
    ON bbb.Match_Id = bs.Match_Id
    AND bbb.Over_Id = bs.Over_Id
    AND bbb.Ball_Id = bs.Ball_Id
LEFT JOIN Extra_Runs er
    ON bbb.Match_Id = er.Match_Id
    AND bbb.Over_Id = er.Over_Id
    AND bbb.Ball_Id = er.Ball_Id
JOIN Player p
    ON bbb.Bowler = p.Player_Id
GROUP BY p.Player_Name, bbb.Match_Id
ORDER BY Wickets DESC, Runs_Conceded ASC
"""

df_best_players_bowlers = pd.read_sql_query(query, conn)
df_best_players_bowlers.head(10)

,Player_Name,Match_Id,Wickets,Runs_Conceded
0,CH Morris,829746,29,150
1,J Theron,419126,22,74
2,R Rampaul,598022,22,119
3,JA Morkel,419126,19,66
4,A Nehra,980984,18,60
5,A Zampa,980984,18,63
6,Z Khan,980956,16,47
7,DJG Sammy,598061,16,52
8,M Muralitharan,419126,16,67
9,UT Yadav,598022,16,90


In [14]:
query = """
WITH Batting_Performance AS (
    SELECT
        p.Player_Id,
        p.Player_Name,
        SUM(bs.Runs_Scored) AS Total_Runs,
        SUM(CASE WHEN bs.Runs_Scored >= 50 AND bs.Runs_Scored < 100 THEN 1 ELSE 0 END) AS Fifties,
        SUM(CASE WHEN bs.Runs_Scored >= 100 THEN 1 ELSE 0 END) AS Centuries
    FROM Batsman_Scored bs
    JOIN Ball_by_Ball bbb ON bs.Match_Id = bbb.Match_Id
                          AND bs.Over_Id = bbb.Over_Id
                          AND bs.Ball_Id = bbb.Ball_Id
    JOIN Player p ON bbb.Striker = p.Player_Id
    GROUP BY p.Player_Id
),
Bowling_Performance AS (
    SELECT
        p.Player_Id,
        COUNT(wt.Player_Out) AS Wickets,
        SUM(COALESCE(bs.Runs_Scored, 0) + COALESCE(er.Extra_Runs, 0)) AS Runs_Conceded
    FROM Ball_by_Ball bbb
    LEFT JOIN Wicket_Taken wt ON bbb.Match_Id = wt.Match_Id
                             AND bbb.Over_Id = wt.Over_Id
                             AND bbb.Ball_Id = wt.Ball_Id
    LEFT JOIN Batsman_Scored bs ON bbb.Match_Id = bs.Match_Id
                               AND bbb.Over_Id = bs.Over_Id
                               AND bbb.Ball_Id = bs.Ball_Id
    LEFT JOIN Extra_Runs er ON bbb.Match_Id = er.Match_Id
                            AND bbb.Over_Id = er.Over_Id
                            AND bbb.Ball_Id = er.Ball_Id
    JOIN Player p ON bbb.Bowler = p.Player_Id
    GROUP BY p.Player_Id
)

SELECT
    bp.Player_Name,
    COALESCE(bat.Total_Runs, 0) AS Total_Runs,
    COALESCE(bat.Fifties, 0) AS Fifties,
    COALESCE(bat.Centuries, 0) AS Centuries,
    COALESCE(bowl.Wickets, 0) AS Wickets,
    COALESCE(bowl.Runs_Conceded, 0) AS Runs_Conceded
FROM Player bp
LEFT JOIN Batting_Performance bat ON bp.Player_Id = bat.Player_Id
LEFT JOIN Bowling_Performance bowl ON bp.Player_Id = bowl.Player_Id
ORDER BY Total_Runs DESC, Wickets DESC;
"""

# Lecture de la requête dans pandas
df_career_stats = pd.read_sql_query(query, conn)
df_career_stats.head(10)

,Player_Name,Total_Runs,Fifties,Centuries,Wickets,Runs_Conceded
0,V Kohli,8158,0,0,32,711
1,SK Raina,7588,0,0,133,2092
2,RG Sharma,7263,0,0,54,915
3,G Gambhir,6793,0,0,0,0
4,RV Uthappa,6303,0,0,0,0
5,DA Warner,6235,0,0,0,0
6,CH Gayle,6105,0,0,88,1567
7,MS Dhoni,6006,0,0,0,0
8,S Dhawan,5917,0,0,12,149
9,AB de Villiers,5832,0,0,0,0
